In [ ]:
# Começamos importando algumas bibliotecas.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from random import sample
from math import sqrt

In [ ]:
# Caso você não possua alguma destas bibliotecas, execute o comando abaixo
!pip install numpy pandas matplotlib scikit-learn --user

# Métodos de agrupamento: K-means

Nesta atividade, vamos implementar um método de agrupamento simples conhecido como K-means (https://en.wikipedia.org/wiki/K-means_clustering). O objetivo do algoritmo é separar um conjunto de pontos em K grupos diferentes. Normalmente você deve conhecer o valor correto de K e passar como parâmetro para o algoritmo. No nosso caso, vamos passar o valor K=5, que corresponde ao número de grupos ancestrais nos dados genéticos apresentados na aula passada: "África, América, Ásia, Europa, Quilombolas".

Você pode ver um exemplo de execução do algoritmo em um conjunto de dados aleatório no vídeo https://www.youtube.com/watch?v=5I3Ei69I40s e interagir com um exemplo interativo em http://stanford.edu/class/ee103/visualizations/kmeans/kmeans.html.

## Passo 1: Carregar o conjunto de dados

Utilizamos o Pandas para carregar o dataframe a partir de um arquivo no Github. Vamos carregar estes dados e remover a coluna de "continentes", já que queremos que o K-means descubra esta informação.

In [ ]:
url = 'https://raw.githubusercontent.com/fabiommendes/desenvolvimento-de-software/master/dados/populations.csv'
dados_brutos = pd.read_csv(url, index_col=0)
continentes = dados_brutos.pop('continent')

display(dados_brutos)

## Passo 2: Limpar o conjunto de dados

Os dados brutos possuem algums elementos NaN (not a number). É necessário atribuir algum valor a estes elementos. Estude a função [fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) do Pandas para descobrir como preencher estes valores com alguma estimativa razoável. 

In [ ]:
# Implemente a estratégia correta de utilizar o fillna. Uma sugestão é preencher
# o valor de cada NaN pela média da sua coluna.
dados = dados_brutos.fillna(...)
dados

In [ ]:
assert dados.dropna().shape == (739, 46)
assert all(tt in (float, int) for tt in dados.dtypes)

## Passo 3: Visualização

Lembra do nosso amigo [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)? Vamos utilizá-lo para visualizar os dados antes de continuar.

In [ ]:
pca = PCA(2).fit(dados)
dados_2d = pca.transform(dados)

X, Y = dados_2d.T
plt.scatter(X, Y)
plt.show()

## Passo 4: Sorteando os centróides iniciais

O K-means tenta encontrar K grupos diferentes, onde cada grupo é caracterizado por um vetor médio (que chamamos centróide) e cada elemento é atribuído ao centróide mais próximo. A inicialização do algoritmo é extremamente simples: basta escolher K pontos aleatoriamente.

Vamos utilizar a função sample do módulo math do Python, já que ela faz exatamente isto:

In [287]:
sample(["foo", "bar", "baz"], 2)

['bar', 'foo']

Adapte o código acima para escolher K = 5 indivíduos aleatoriamente da população completa. Você provavelmente terá que converter a população para um array numpy utilizando `dados.values`. 

In [ ]:
K = 5

def sorteia_centroides(elementos, k):
    """
    Escolhe aleatoriamente k elementos da lista fornecida.
    """
    return sample(...)


sorteia_centroides(dados.values, K)

In [ ]:
assert sorteia_centroides([(1, 1), (1, 1)], 1) == [(1, 1)]

## Passo 5: Calculando a distância

O próximo passo consiste em determinar a distância de cada elemento para o centróide, para podermos agrupar cada ponto no seu respectivo grupo. Começamos com um objetivo um pouco mais modesto: calcular a distância entre dois pontos.

Lembre-se da fórmula para dois vetores $u$, e $v$ de $n$ dimensões. 

$$d(\vec u, \vec v) = \sqrt{\sum_{i=0}^{n} \left(u_i - v_i \right)^{2}}$$


In [ ]:
# IMPORTANTE: sua fórmula deve funcionar corretamente para vetores 
# de qualquer número de componentes

def distancia(x, y):
    # converte para array, habilitando operações matemáticas 
    # com x e y
    x = np.asarray(x)
    y = np.asarray(y)
    return ...

print('Distância entre vetores:', distancia((0, 3), (4, 0)))

In [ ]:
assert distancia([0, 3], [4, 0]) == 5.0
assert distancia(dados.values[0], dados.values[0]) == 0
assert abs(distancia(dados.values[0], dados.values[1]) - 4.6904) < 1e-3

## Passso 6: encontrando o centroide mais próximo

O passo seguinte no PCA consiste em atribuir cada elemento a um centróide, sempre escolhendo o centróide mais próximo. Nós vamos fazer isto criando uma função que passa pela lista de elementos e retorna uma lista de listas onde cada uma destas listas.

Antes de fazer isto, implementamos a função `mais_proximo` abaixo.

In [ ]:
def mais_proximo(x, centros):
    """
    Retorna o índice do centroide mais próximo do elemento
    x na lista de centros dada. 
    """
    
    menor_idx = ...
    menor_distancia = ...
    
    for i, centro in enumerate(centros):
        d = distancia(x, centro)
        ...
        
    return menor_idx
    

# Exemplo: o ponto (1, 2) está mais próximo de (1, 1) nos centroides abaixo
mais_proximo((1, 2), [(0, 0), (1, 1), (5, 5), (-3, 4)])

In [ ]:
assert mais_proximo((1, 2), [(0, 0), (1, 1), (5, 5), (-3, 4)]) == 1
assert mais_proximo(dados.values[1], dados.values[:5]) == 1

## Passo 7: Atribuindo grupos

Agora vamos utilizar a nossa função `mais_proximo` para realmente distribuir os elementos em seus respectivos grupos. Implemente a função `atribui_grupos` abaixo.

In [ ]:
def atribui_grupos(elementos, centros):
    """
    Distribui os elementos da lista dada entre os seus respectivos
    centroides. O resultado são K listas diferentes (onde K corresponde
    ao número de centróides). 
    
    Cada elemento é atribuído à lista com o K mais próximo.
    """
    grupos = [[] for _ in centros]
    
    for x in elementos:
        idx = mais_proximo(x, centros)
        grupos[idx].append(x)
    return grupos

# Exemplo: 2 centroides
centros = [(1, 2), (-1, -2)]
pontos = [(1, 1), (2, 2), (-5, -4), (0, 2), (-1, -1)]
g1, g2 = atribui_grupos(pontos, centros)
print('Grupo 1:', g1)
print('Grupo 2:', g2)

In [ ]:
assert atribui_grupos([(-1, -1), (1, 1)], [(1, 2), (-1, -2)]) == [[(1, 1)], [(-1, -1)]]
assert [len(g) for g in atribui_grupos(dados.values, dados.values[:5])] == [40, 84, 11, 367, 237]

## Passo 8: Calculando o centroide

Agora temos uma tarefa bem simples: dado um grupo de pontos, devemos calcular o centróide para fazer a atualização do k-means. Implemente a função centroide que calcula justamente isto.

In [ ]:
def centroide(grupo):
    """
    Calcula o centroide (média) de um único grupo de elementos.
    """
    # Converta cada elemento para array para
    # simplificar o cálculo :)
    elems = [np.asarray(x) for x in grupo]
    return ...


def centroides(grupos):
    """
    Aplica centroide em cada grupo na lista de grupos e
    retorna uma lista de centroides.
    """
    return [centroide(g) for g in grupos]


# Exemplo: 2 centroides
grupos = [[(1, 1), (2, 2), (0, 2)], [(-5, -4), (-1, -1)]]
centroides(grupos)

In [ ]:
assert all(np.all(x == y)
           for x, y in zip(centroides([[(1, 1), (2, 2)], [(-5, -4), (-1, -1)]]),
                           [np.array([1.5, 1.5]), np.array([-3. , -2.5])]))

## Passo 9: Medindo a variação

Uma maneira de formalizar o que o K-means está fazendo é entender o algoritmo como um processo de minimização. A discussão matemática completa está além do objetivo deste tutorial. De todo jeito, os matemáticos descobriram que o K-means é um processo que minimiza a variação quadrática em cada etapa do algoritmo. A variação quadrática é a soma dos quadrados das distâncias de cada elemento para o seu respectivo centro. 

Intuitivamente, podemos pensar que uma variação quadrática pequena corresponde a situação ideal onde cada elemento está muito próximo do seu respectivo centro e portanto foi corretamente classificado. Implemente a função VQ que calcula a variação quadrática do conjunto de dados. A variação quadrática sempre diminui ou se mantêm constante em cada etapa do k-means.

In [ ]:
def vq(grupo):
    """Calcula a variação quadrática"""
    centro = centroide(grupo)
    return ...

def vq_total(grupos):
    """Calcula a variação quadrática de todos grupos somados"""
    return sum(vq(g) for g in grupos)

# Exemplo
vq([(0, 0), (1, 1)])  # vq = 1, porque?

In [ ]:
assert abs(vq([(0, 0), (1, 1)]) - 1) < 1e-3
assert abs(vq(dados.values[:100]) - 1292.65) < 1e-1

## Passo 10: Iteração completa

Vamos realizar uma iteração completa do algoritmo. Isto corresponde aos passos 
"encontrar centroides mais próximos" e depois "atualizar centroides" da simulação em http://stanford.edu/class/ee103/visualizations/kmeans/kmeans.html. Observe que a variação quadrática sempre diminui quando fazemos uma iteração. Podemos parar o algortimo de k-means quando a variação quadrática parar de se alterar.

In [ ]:
k = 5
centros = sorteia_centroides(dados.values, k)

In [ ]:
# Execute esta célula várias vezes para ver o VQ diminuir
grupos = atribui_grupos(dados.values, centros)
centros = centroides(grupos)
print('VQ:', vq_total(grupos))

## Passo 11: juntando as partes

Vamos colocar cada um destes passos dentro de um loop e executar o algoritmo enquanto
a variação quadrática se alterar.

In [ ]:
def kmeans(dados, k=4):
    """
    Calcula o K-means para o conjunto de dados.
    """
    
    vq_inicial = ...
    centros = sorteia_centroides(dados, k)
    
    while True:
        grupos = atribui_grupos(dados, centros)
        centros = centroides(grupos)
        print('vq:', vq_total(grupos))

        ... # Implemente o critério de parada
            
    # Vamos converter cada grupo para um array 2d ao
    # invés de uma lista de arrays
    return [np.asarray(g) for g in grupos]


# Vamos visualizar os dados com PCA.
pca = PCA(2).fit(dados)

for group in kmeans(dados.values, 5):
    X, Y = pca.transform(group).T
    plt.scatter(X, Y, alpha=0.6)
plt.show()

## Parte 12: Exemplo

Aqui montamos um exemplo com dados fictícios.

In [ ]:
std = .75
g1 = np.random.normal((0, 0), std, size=(500, 2))
g2 = np.random.normal((3, 4), std, size=(300, 2))
g3 = np.random.normal((3, 0), std, size=(200, 2))
completo = np.vstack([g1, g2, g3])

X, Y = completo.T
plt.scatter(X, Y, alpha=0.5)
plt.title('Pontos gerados artificialmente')
plt.show()

In [ ]:
K = 3
for grupo in kmeans(completo, K):
    X, Y = grupo.T
    plt.scatter(X, Y, alpha=0.5)
plt.title('K-means')
plt.show()